In [1]:
# 동영상 촬영 및 녹화, 저장_ (streamlit x)

import cv2

# 저장할 파일 이름 및 코덱 설정
output_filename = "interview_video.avi"
fps = 20.0
frame_size = (640, 480)  # 영상 크기 (가로, 세로)

# VideoWriter 객체 생성
fourcc = cv2.VideoWriter_fourcc(*'XVID')  # 코덱: XVID
out = cv2.VideoWriter(output_filename, fourcc, fps, frame_size)

# 카메라 열기 (기본 장치: 0)
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("웹캠을 열 수 없습니다.")
    exit()

print("녹화를 시작합니다. 종료하려면 'q' 키를 누르세요.")

while True:
    ret, frame = cap.read()
    if not ret:
        print("프레임을 읽을 수 없습니다. 종료합니다.")
        break

    # 화면에 보여주기
    cv2.imshow('Recording...', frame)

    # 파일로 저장
    out.write(frame)

    # 종료 조건: q 키
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 리소스 해제
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"녹화가 완료되었습니다. 파일 저장 위치: {output_filename}")

녹화를 시작합니다. 종료하려면 'q' 키를 누르세요.
녹화가 완료되었습니다. 파일 저장 위치: interview_video.avi


In [2]:
# Mediapipe로 얼굴 감정용 랜드마크 추적 _ (streamlit x)

import cv2
import mediapipe as mp

mp_face_mesh = mp.solutions.face_mesh
cap = cv2.VideoCapture(0)

with mp_face_mesh.FaceMesh(max_num_faces=1, refine_landmarks=True) as face_mesh:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = face_mesh.process(img_rgb)

        if result.multi_face_landmarks:
            for face_landmarks in result.multi_face_landmarks:
                mp.solutions.drawing_utils.draw_landmarks(
                    frame, face_landmarks, mp_face_mesh.FACEMESH_TESSELATION)

        cv2.imshow("FaceMesh Live", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [6]:
# mediapipe face mesh -> 나오는 값으로 눈동자 -> 랜드마크 간 벡터 계산표정 감지 (웃음, 긴장 등)/	입꼬리, 눈꼬리, 턱 위치 분석
import cv2
import mediapipe as mp
import numpy as np

# 설정
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(max_num_faces=1)
mp_draw = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)

while True:
    ret, img = cap.read()
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(img_rgb)

    if results.multi_face_landmarks:
        for landmarks in results.multi_face_landmarks:
            # 눈, 입꼬리, 눈썹 등 특정 좌표 인덱스 추출
            h, w, _ = img.shape

            # 예: 왼쪽 눈동자 위치 (468 포인트 중 일부)
            left_eye_inner = landmarks.landmark[133]
            right_eye_inner = landmarks.landmark[362]
            nose_tip = landmarks.landmark[1]
            mouth_left = landmarks.landmark[61]
            mouth_right = landmarks.landmark[291]

            # 좌표 변환
            lx, ly = int(left_eye_inner.x * w), int(left_eye_inner.y * h)
            rx, ry = int(right_eye_inner.x * w), int(right_eye_inner.y * h)

            # 시선 방향 간단 판별 (눈동자 간 거리 등 비교)
            eye_dir = '정면'
            if lx - rx > 15:
                eye_dir = '왼쪽 봄'
            elif rx - lx > 15:
                eye_dir = '오른쪽 봄'

            # 입꼬리 높이 비교 → 웃음 여부 추정
            mx1 = int(mouth_left.y * h)
            mx2 = int(mouth_right.y * h)
            smile = abs(mx1 - mx2) < 5  # 웃을 땐 좌우 입꼬리 높이가 비슷

            # 피드백 출력
            feedback = f"{eye_dir}, {'웃음' if smile else '무표정'}"
            cv2.putText(img, feedback, (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)

            mp_draw.draw_landmarks(img, landmarks, mp_face_mesh.FACEMESH_CONTOURS)

    cv2.imshow("Live Feedback", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [7]:
# mediapipe face mesh + 한글 폰트
import cv2
import mediapipe as mp
import numpy as np
from PIL import ImageFont, ImageDraw, Image

# -----------------------------
# [2단계] 한글 텍스트 출력 함수 정의
# -----------------------------
def draw_hangul_text(img, text, position, font_path, font_size=32, color=(0, 255, 0)):
    img_pil = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(img_pil)
    font = ImageFont.truetype(font_path, font_size)
    draw.text(position, text, font=font, fill=color)
    return cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)

# -----------------------------
# [1단계] FaceMesh 초기화
# -----------------------------
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(max_num_faces=1)
mp_drawing = mp.solutions.drawing_utils

# -----------------------------
# 웹캠 스트리밍 시작
# -----------------------------
cap = cv2.VideoCapture(0)
font_path = "C:/Windows/Fonts/malgun.ttf"  # 한글 폰트 경로

while cap.isOpened():
    success, img = cap.read()
    if not success:
        break

    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(img_rgb)
    h, w, _ = img.shape

    feedback_text = "감지 안됨"

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            mp_drawing.draw_landmarks(img, face_landmarks, mp_face_mesh.FACEMESH_TESSELATION,
                                      landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=1, circle_radius=1))

            # 좌표 추출
            left_eye_inner = face_landmarks.landmark[133]
            right_eye_inner = face_landmarks.landmark[362]
            mouth_left = face_landmarks.landmark[61]
            mouth_right = face_landmarks.landmark[291]

            lx, ly = int(left_eye_inner.x * w), int(left_eye_inner.y * h)
            rx, ry = int(right_eye_inner.x * w), int(right_eye_inner.y * h)
            mx1 = int(mouth_left.y * h)
            mx2 = int(mouth_right.y * h)

            # -----------------------------
            # [3단계] 시선 방향 추정
            # -----------------------------
            eye_dir = "정면 응시"
            if lx - rx > 15:
                eye_dir = "왼쪽 응시"
            elif rx - lx > 15:
                eye_dir = "오른쪽 응시"

            # -----------------------------
            # 입꼬리 위치로 웃음 여부 추정
            # -----------------------------
            smile = abs(mx1 - mx2) < 5
            smile_text = "미소" if smile else "무표정"

            # -----------------------------
            # 피드백 한글 텍스트 생성
            # -----------------------------
            feedback_text = f"{eye_dir}, {smile_text}"

    # -----------------------------
    # 한글 텍스트 이미지에 출력
    # -----------------------------
    img = draw_hangul_text(img, feedback_text, (30, 50), font_path)

    cv2.imshow("Live Feedback", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [2]:
!pip uninstall opencv-python opencv-contrib-python -y

In [3]:
!pip install opencv-python opencv-contrib-python --no-cache-dir --force-reinstall

   ---------------------------------------- 0.0/39.0 MB ? eta -:--:--
   -- ------------------------------------- 2.6/39.0 MB 12.6 MB/s eta 0:00:03
   ----- ---------------------------------- 5.2/39.0 MB 12.8 MB/s eta 0:00:03
   ------- -------------------------------- 7.1/39.0 MB 11.2 MB/s eta 0:00:03
   --------- ------------------------------ 9.7/39.0 MB 11.6 MB/s eta 0:00:03
   ------------ --------------------------- 12.3/39.0 MB 11.9 MB/s eta 0:00:03
   -------------- ------------------------- 14.2/39.0 MB 11.4 MB/s eta 0:00:03
   ---------------- ----------------------- 16.5/39.0 MB 11.4 MB/s eta 0:00:02
   ------------------- -------------------- 19.1/39.0 MB 11.5 MB/s eta 0:00:02
   ---------------------- ----------------- 21.8/39.0 MB 11.7 MB/s eta 0:00:02
   ------------------------- -------------- 24.4/39.0 MB 11.8 MB/s eta 0:00:02
   --------------------------- ------------ 26.7/39.0 MB 11.8 MB/s eta 0:00:02
   ------------------------------ --------- 29.4/39.0 MB 11.8 MB/

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mediapipe 0.10.21 requires numpy<2, but you have numpy 2.2.6 which is incompatible.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 2.2.6 which is incompatible.


In [4]:
!pip install numpy==1.26.4 --force-reinstall

  Using cached numpy-1.26.4-cp311-cp311-win_amd64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-win_amd64.whl (15.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-2.2.6:
      Successfully uninstalled numpy-2.2.6


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.


In [5]:
!pip uninstall opencv-python opencv-contrib-python numpy -y

Found existing installation: opencv-python 4.12.0.88
Uninstalling opencv-python-4.12.0.88:
  Successfully uninstalled opencv-python-4.12.0.88
Found existing installation: opencv-contrib-python 4.12.0.88
Uninstalling opencv-contrib-python-4.12.0.88:
  Successfully uninstalled opencv-contrib-python-4.12.0.88
Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4


In [6]:
!pip install deepface --no-cache-dir --force-reinstall

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 1.4/1.4 MB 8.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ---------------------------------------- 1.9/1.9 MB 11.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/39.0 MB ? eta -:--:--
   -- ------------------------------------- 2.4/39.0 MB 12.3 MB/s eta 0:00:03
   ----- ---------------------------------- 5.0/39.0 MB 12.1 MB/s eta 0:00:03
   -------- ------------------------------- 7.9/39.0 MB 12.5 MB/s eta 0:00:03
   ---------- ----------------------------- 10.2/39.0 MB 12.3 MB/s eta 0:00:03
   ------------- -------------------------- 12.8/39.0 MB 12.4 MB/s eta 0:00:03
   --------------- ------------------------ 15.5/39.0 MB 12.5 MB/s eta 0:00:02
   ------------------ --------------------- 17.8/39.0 MB 12.5 MB/s

  DEPRECATION: Building 'fire' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'fire'. Discussion can be found at https://github.com/pypa/pip/issues/6334
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mediapipe 0.10.21 requires opencv-contrib-python, which is not installed.
mediapipe 0.10.21 requires numpy<2, but you have numpy 2.1.3 which is incompatible.
mediapipe 0.10.21 requires protobuf<5,>=4.25.3, but you have protobuf 5.29.5 which is incompatible.


In [ ]:
pip install numpy==1.26.4


In [ ]:
pip install opencv-python==4.7.0.72


In [ ]:
pip install opencv-contrib-python==4.7.0.72


In [ ]:
pip install tensorflow==2.19.0


In [ ]:
pip install mediapipe==0.10.21


In [ ]:
pip install deepface

In [3]:
# 시선 추적 및 표전 분석용 모델 적용 _ emotion_model.h5 없음

import cv2
import mediapipe as mp
import numpy as np
import tensorflow as tf
import time

# 🧠 감정 라벨 정의
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

# 🧠 학습된 CNN 모델 불러오기
emotion_model = tf.keras.models.load_model('emotion_model.h5')

# 🎯 Face Mesh 설정
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False,
                                   refine_landmarks=True,
                                   max_num_faces=1,
                                   min_detection_confidence=0.5,
                                   min_tracking_confidence=0.5)

# 🎯 MediaPipe 얼굴 감지용 (표정 분석용)
mp_face_detection = mp.solutions.face_detection
face_detection = mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.5)

# 👁️ 눈 및 동공 좌표 인덱스
LEFT_EYE = [33, 133]
RIGHT_EYE = [362, 263]
LEFT_IRIS = [468, 469, 470, 471]
RIGHT_IRIS = [473, 474, 475, 476]

def calc_gaze_ratio(iris, eye):
    center = np.mean(iris, axis=0)
    left, right = eye
    return (center[0] - left[0]) / (right[0] - left[0])

def get_emotion(face_img):
    gray = cv2.cvtColor(face_img, cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(gray, (48, 48)) / 255.0
    input_img = resized.reshape(1, 48, 48, 1)
    prediction = emotion_model.predict(input_img, verbose=0)
    return emotion_labels[np.argmax(prediction)]

# 🎥 카메라 시작
cap = cv2.VideoCapture(0)
prev_time = time.time()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv2.flip(frame, 1)
    h, w = frame.shape[:2]
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # 시선 분석
    results = face_mesh.process(rgb)
    gaze_text = "시선 감지 중..."

    if results.multi_face_landmarks:
        landmarks = results.multi_face_landmarks[0].landmark

        def extract(index_list):
            return np.array([[int(landmarks[i].x * w), int(landmarks[i].y * h)] for i in index_list])

        left_eye = extract(LEFT_EYE)
        right_eye = extract(RIGHT_EYE)
        left_iris = extract(LEFT_IRIS)
        right_iris = extract(RIGHT_IRIS)

        left_ratio = calc_gaze_ratio(left_iris, left_eye)
        right_ratio = calc_gaze_ratio(right_iris, right_eye)
        avg_ratio = (left_ratio + right_ratio) / 2

        if avg_ratio < 0.25:
            gaze_text = "시선: 왼쪽 화면 밖"
        elif avg_ratio > 0.75:
            gaze_text = "시선: 오른쪽 화면 밖"
        else:
            gaze_text = "시선: 화면 중앙"

    # 표정 감정 분석 (0.2초 간격)
    now = time.time()
    if now - prev_time >= 0.2:
        emotion_text = "표정 분석 중..."
        results_fd = face_detection.process(rgb)
        if results_fd.detections:
            for detection in results_fd.detections:
                bboxC = detection.location_data.relative_bounding_box
                x, y = int(bboxC.xmin * w), int(bboxC.ymin * h)
                w_box, h_box = int(bboxC.width * w), int(bboxC.height * h)
                face_img = frame[y:y+h_box, x:x+w_box]
                if face_img.size > 0:
                    emotion_text = get_emotion(face_img)
                    cv2.putText(frame, f'감정: {emotion_text}', (x, y-10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 200, 0), 2)
                    cv2.rectangle(frame, (x, y), (x+w_box, y+h_box), (255, 200, 0), 2)
        prev_time = now

    # 결과 시각화
    cv2.putText(frame, gaze_text, (30, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    cv2.imshow("시선 및 표정 분석", frame)

    if cv2.waitKey(1) & 0xFF == 27:  # ESC 키 종료
        break

cap.release()
cv2.destroyAllWindows()


FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'emotion_model.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [10]:
!pip install deepface

  Using cached opencv_python-4.12.0.88-cp37-abi3-win_amd64.whl.metadata (19 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached numpy-2.2.6-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Using cached numpy-2.1.3-cp311-cp311-win_amd64.whl.metadata (60 kB)
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ---------------------------------------- 1.9/1.9 MB 10.5 MB/s eta 0:00:00
Using cached opencv_python-4.12.0.88-cp37-abi3-win_amd64.whl (39.0 MB)
Using cached numpy-2.1.3-cp311-cp311-win_amd64.whl (12.9 MB)
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114349 sha256=ff227dcd1eebdd77cdbbcb272ffa4fccbddb19c416548b46bd15cd49b6b7b766
  Stored in directory: c:\users\mira\appdata\local\pip\cache\wheels\46\54\24\1624fd5b8674eb1188623f7e8e17cdf7c0f6c24b609dfb8a89
Successfully built fire

  Attempting uninstall: numpy

    Found existing installation: numpy 1.26.4

   ----- -------------

  DEPRECATION: Building 'fire' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'fire'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  You can safely remove it manually.
  You can safely remove it manually.
ERROR: Could not install packages due to an OSError: [WinError 5] 액세스가 거부되었습니다: 'C:\\Users\\Mira\\miniconda3\\envs\\interview_env\\Lib\\site-packages\\cv2\\cv2.pyd'
Consider using the `--user` option or check the permissions.



In [9]:
# 시선 추적 및 표전 분석용 모델 적용 _ emotion_model.h5 없어서 deepface 로 변경
import cv2
import mediapipe as mp
import numpy as np
import time
from deepface import DeepFace  # ✅ DeepFace 추가

# 👁️ 눈 및 동공 좌표 인덱스
LEFT_EYE = [33, 133]
RIGHT_EYE = [362, 263]
LEFT_IRIS = [468, 469, 470, 471]
RIGHT_IRIS = [473, 474, 475, 476]

# 🎯 Face Mesh 설정
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False,
                                   refine_landmarks=True,
                                   max_num_faces=1,
                                   min_detection_confidence=0.5,
                                   min_tracking_confidence=0.5)

# 🎯 얼굴 감지용
mp_face_detection = mp.solutions.face_detection
face_detection = mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.5)

# 🧠 시선 계산
def calc_gaze_ratio(iris, eye):
    center = np.mean(iris, axis=0)
    left, right = eye
    return (center[0] - left[0]) / (right[0] - left[0])

# ✅ DeepFace 기반 감정 분석 함수
def get_emotion(face_img):
    try:
        result = DeepFace.analyze(face_img, actions=['emotion'], enforce_detection=False)
        emotion = result[0]['dominant_emotion']
        return emotion
    except Exception as e:
        print("감정 분석 오류:", e)
        return "분석 실패"

# 🎥 카메라 시작
cap = cv2.VideoCapture(0)
prev_time = time.time()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv2.flip(frame, 1)
    h, w = frame.shape[:2]
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # 시선 분석
    results = face_mesh.process(rgb)
    gaze_text = "시선 감지 중..."

    if results.multi_face_landmarks:
        landmarks = results.multi_face_landmarks[0].landmark

        def extract(index_list):
            return np.array([[int(landmarks[i].x * w), int(landmarks[i].y * h)] for i in index_list])

        left_eye = extract(LEFT_EYE)
        right_eye = extract(RIGHT_EYE)
        left_iris = extract(LEFT_IRIS)
        right_iris = extract(RIGHT_IRIS)

        left_ratio = calc_gaze_ratio(left_iris, left_eye)
        right_ratio = calc_gaze_ratio(right_iris, right_eye)
        avg_ratio = (left_ratio + right_ratio) / 2

        if avg_ratio < 0.25:
            gaze_text = "시선: 왼쪽 화면 밖"
        elif avg_ratio > 0.75:
            gaze_text = "시선: 오른쪽 화면 밖"
        else:
            gaze_text = "시선: 화면 중앙"

    # 감정 분석 (0.5초 간격)
    now = time.time()
    if now - prev_time >= 0.5:
        emotion_text = "표정 분석 중..."
        results_fd = face_detection.process(rgb)
        if results_fd.detections:
            for detection in results_fd.detections:
                bboxC = detection.location_data.relative_bounding_box
                x, y = int(bboxC.xmin * w), int(bboxC.ymin * h)
                w_box, h_box = int(bboxC.width * w), int(bboxC.height * h)
                face_img = frame[y:y+h_box, x:x+w_box]
                if face_img.size > 0:
                    emotion = get_emotion(face_img)
                    cv2.putText(frame, f'감정: {emotion}', (x, y-10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 200, 0), 2)
                    cv2.rectangle(frame, (x, y), (x+w_box, y+h_box), (255, 200, 0), 2)
        prev_time = now

    # 결과 시각화
    cv2.putText(frame, gaze_text, (30, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    cv2.imshow("시선 및 표정 분석", frame)

    if cv2.waitKey(1) & 0xFF == 27:  # ESC
        break

cap.release()
cv2.destroyAllWindows()

ModuleNotFoundError: No module named 'deepface'

In [ ]:
# 그라디오 잘 작동하는지 test 해봄

import gradio as gr

def greet(name):
    return f"안녕하세요, {name}님!"

iface = gr.Interface(fn=greet, inputs="text", outputs="text")

try:
    iface.launch(share=True)
except Exception as e:
    print("[Gradio 런타임 에러 발생]")
    print(e)

In [6]:
# gradio, 시선추적, 점수, 표정분석, 음성 받기

import os
import cv2
import numpy as np
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import mediapipe as mp
import gradio as gr  # moved after successful imports

# ---------------------
# 1. PyTorch 모델 정의 및 로드
# ---------------------
EMOTIONS = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

class EmotionCNN(nn.Module):
    def __init__(self):
        super(EmotionCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 12 * 12, 128), nn.ReLU(),
            nn.Linear(128, len(EMOTIONS))
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

model_path = "emotion_model.pt"
if not os.path.exists(model_path):
    raise FileNotFoundError(f"모델 파일 '{model_path}'이(가) 존재하지 않습니다. 파일을 확인해주세요.")

model = EmotionCNN()
model.load_state_dict(torch.load(model_path, map_location='cpu'))
model.eval()

# ---------------------
# 2. 전처리 도구 및 설정
# ---------------------
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

mp_face = mp.solutions.face_mesh
face_mesh = mp_face.FaceMesh(refine_landmarks=True, max_num_faces=1)

# ---------------------
# 3. 실시간 점수 상태 변수
# ---------------------
emotion_score = 0
gaze_score = 0
frame_counter = 0

# ---------------------
# 4. 실시간 분석 함수
# ---------------------
def analyze(frame):
    global emotion_score, gaze_score, frame_counter

    frame_counter += 1
    display = frame.copy()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 얼굴 인식 + 감정 분석
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x, y, w, h) in faces:
        roi = gray[y:y+h, x:x+w]
        roi_resized = cv2.resize(roi, (48, 48))
        input_tensor = transform(roi_resized).unsqueeze(0)
        with torch.no_grad():
            output = model(input_tensor)
            pred = torch.argmax(output, dim=1).item()
            label = EMOTIONS[pred]
            emotion_score += pred

        cv2.rectangle(display, (x, y), (x+w, y+h), (0,255,0), 2)
        cv2.putText(display, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2)

    # 시선 응시 판단
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = face_mesh.process(img_rgb)

    if result.multi_face_landmarks:
        h, w, _ = frame.shape
        face = result.multi_face_landmarks[0]
        left_eye = face.landmark[145]
        right_eye = face.landmark[374]
        x_mid = int((left_eye.x + right_eye.x) / 2 * w)
        if abs(x_mid - w // 2) < 80:
            gaze_score += 1
            cv2.putText(display, "✅ 응시 중", (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
        else:
            cv2.putText(display, "❌ 응시 아닌", (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)

    # 점수 출력 (로그)
    if frame_counter % 150 == 0:
        avg_emotion = emotion_score / frame_counter
        avg_gaze = gaze_score / frame_counter
        print(f"[INFO] 감정 평균: {avg_emotion:.2f}, 응시률: {avg_gaze:.2f}")

    return display

# ---------------------
# 5. Gradio 앱 실행
# ---------------------
try:
    gr.Interface(
        fn=analyze,
        inputs=gr.Image(source="webcam", streaming=True),
        outputs="image",
        title="🌟 실시간 감정 분석 + 시선 응시 평가",
        live=True
    ).launch()
except ModuleNotFoundError as e:
    print("[ERROR] 필요한 모듈이 누락되었습니다:", e)
    print("pip install gradio opencv-python torch torchvision mediapipe numpy 등을 설치해주세요.")

FileNotFoundError: 모델 파일 'emotion_model.pt'이(가) 존재하지 않습니다. 파일을 확인해주세요.

In [1]:
# gradio, 시선추적, 점수, 표정분석, 음성 받기 -> deepface로 감정 분석으로 바꿈
import os
import cv2
import numpy as np
import mediapipe as mp
from deepface import DeepFace
import gradio as gr

EMOTIONS = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
mp_face = mp.solutions.face_mesh
face_mesh = mp_face.FaceMesh(refine_landmarks=True, max_num_faces=1)

emotion_score = 0
gaze_score = 0
frame_counter = 0

def analyze(frame):
    global emotion_score, gaze_score, frame_counter

    frame_counter += 1
    display = frame.copy()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x, y, w, h) in faces:
        roi_color = frame[y:y+h, x:x+w]
        try:
            result = DeepFace.analyze(roi_color, actions=['emotion'], enforce_detection=False)
            label = result[0]['dominant_emotion']
            emotion_idx = EMOTIONS.index(label) if label in EMOTIONS else 0
            emotion_score += emotion_idx
        except Exception as e:
            label = "Unknown"
            print(f"[DeepFace ERROR]: {e}")
        cv2.rectangle(display, (x, y), (x+w, y+h), (0,255,0), 2)
        cv2.putText(display, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2)

    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = face_mesh.process(img_rgb)

    if result.multi_face_landmarks:
        h, w, _ = frame.shape
        face = result.multi_face_landmarks[0]
        left_eye = face.landmark[145]
        right_eye = face.landmark[374]
        x_mid = int((left_eye.x + right_eye.x) / 2 * w)
        if abs(x_mid - w // 2) < 80:
            gaze_score += 1
            cv2.putText(display, "✅ 응시 중", (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
        else:
            cv2.putText(display, "❌ 응시 아닌", (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)

    if frame_counter % 150 == 0:
        avg_emotion = emotion_score / frame_counter
        avg_gaze = gaze_score / frame_counter
        print(f"[INFO] 감정 평균(인덱스): {avg_emotion:.2f}, 응시률: {avg_gaze:.2f}")

    return display

# Gradio 3.50.2 방식
gr.Interface(
    fn=analyze,
    inputs=gr.Image(source="webcam", streaming=True),
    outputs="image",
    title="🌟 실시간 감정 분석 (DeepFace) + 시선 응시 평가",
    live=True
).launch()


IMPORTANT: You are using gradio version 3.23.0, however version 4.44.1 is available, please upgrade.
--------
Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [3]:
!pip install streamlit-webrtc==0.45.0

   ---------------------------------------- 0.0/872.3 kB ? eta -:--:--
   ---------------------------------------- 872.3/872.3 kB 9.8 MB/s eta 0:00:00
  Attempting uninstall: streamlit-webrtc
    Found existing installation: streamlit-webrtc 0.63.3
    Uninstalling streamlit-webrtc-0.63.3:
      Successfully uninstalled streamlit-webrtc-0.63.3


In [1]:
import cv2
import mediapipe as mp
import time

# MediaPipe 얼굴 메쉬 초기화
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

# Drawing 유틸리티
mp_drawing = mp.solutions.drawing_utils
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)

# OpenCV 웹캠 시작
cap = cv2.VideoCapture(0)

while cap.isOpened():
    start_time = time.time()  # 프레임 시작 시간 기록
    success, image = cap.read()
    if not success:
        print("카메라에서 프레임을 가져올 수 없습니다.")
        break

    # 성능을 위해 이미지 읽기 전 BGR → RGB 변환
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_rgb.flags.writeable = False

    # MediaPipe 얼굴 메쉬 추론
    results = face_mesh.process(image_rgb)

    # 다시 BGR로 변환 후 그리기 위해 writeable 설정
    image.flags.writeable = True
    image = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)

    # 얼굴 랜드마크 그리기
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            mp_drawing.draw_landmarks(
                image=image,
                landmark_list=face_landmarks,
                connections=mp_face_mesh.FACEMESH_TESSELATION,
                landmark_drawing_spec=None,
                connection_drawing_spec=drawing_spec
            )
            # print(results)
            print(results.multi_face_landmarks)
            

    ##### 시간 계산  ##############
    end_time = time.time()  # 프레임 처리 끝난 시간 기록
    elapsed_time = end_time - start_time  # 처리 시간
    fps = 1 / elapsed_time if elapsed_time > 0 else 0

    # 시간 표시
    cv2.putText(image, f"Frame Time: {elapsed_time:.3f}s | FPS: {fps:.1f}",
                (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    cv2.imshow('Frame Time Test', image)

    break

    #################################
            

    # 결과 화면 출력
    cv2.imshow('MediaPipe FaceMesh', image)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 종료 처리
cap.release()
cv2.destroyAllWindows()

[landmark {
  x: 0.514542103
  y: 0.808864415
  z: -0.0483120568
}
landmark {
  x: 0.517623
  y: 0.761680245
  z: -0.0771900937
}
landmark {
  x: 0.515282333
  y: 0.783927321
  z: -0.0432412922
}
landmark {
  x: 0.503980041
  y: 0.697004199
  z: -0.0548243523
}
landmark {
  x: 0.517984
  y: 0.740835547
  z: -0.0807978958
}
landmark {
  x: 0.517355204
  y: 0.714984655
  z: -0.0733409673
}
landmark {
  x: 0.514769197
  y: 0.654664516
  z: -0.0301669035
}
landmark {
  x: 0.407789171
  y: 0.642181635
  z: 0.0182829481
}
landmark {
  x: 0.513938904
  y: 0.608848631
  z: -0.0176640637
}
landmark {
  x: 0.514049
  y: 0.583316207
  z: -0.0184821244
}
landmark {
  x: 0.513586581
  y: 0.492643893
  z: 0.00569298444
}
landmark {
  x: 0.514871061
  y: 0.8194471
  z: -0.04703914
}
landmark {
  x: 0.515004516
  y: 0.832656145
  z: -0.042534627
}
landmark {
  x: 0.515028119
  y: 0.843196452
  z: -0.0356104299
}
landmark {
  x: 0.515159249
  y: 0.849719286
  z: -0.0344261713
}
landmark {
  x: 0.515739

In [1]:
!pip install streamlit-webrtc==0.45.0 --no-cache-dir

   ---------------------------------------- 0.0/872.3 kB ? eta -:--:--
   ------------------------ --------------- 524.3/872.3 kB 8.5 MB/s eta 0:00:01
   ------------------------ --------------- 524.3/872.3 kB 8.5 MB/s eta 0:00:01
   ---------------------------------------- 872.3/872.3 kB 1.4 MB/s eta 0:00:00
